In [33]:
import pandas as pd

In [34]:
cf_test_reco = pd.read_excel('../data/results/cf_test_reco.xlsx')
one_to_one_test_reco = pd.read_excel('../data/results/onetoone_test_reco.xlsx')
nonactive_test_reco = pd.read_excel('../data/results/nonactive_test_reco.xlsx')

cf_holdout_reco = pd.read_excel('../data/results/cf_holdout_reco.xlsx')
one_to_one_holdout_reco = pd.read_excel('../data/results/onetoone_holdout_reco.xlsx')
nonactive_holdout_reco = pd.read_excel('../data/results/nonactive_holdout_reco.xlsx')

cf_holdout_would_have = pd.read_excel('../data/results/cf_holdout_would_have_reco.xlsx')
one_to_one_holdout_would_have = pd.read_excel('../data/results/onetoone_holdout_would_have_reco.xlsx')

In [35]:
cf_test_reco.head()

,input_buyer_nbr,original_lot,recommended_lot,manhattan_distance,source
0,557,84058905,81991085,3826.74,Step 1 - YMM/Manhattan
1,557,68938855,68839585,2097.52,Step 1 - YMM/Manhattan
2,557,84976465,86972465,146.81,Step 1 - YMM/Manhattan
3,557,87056895,81188895,983.00,Step 1 - YMM/Manhattan
4,557,71597865,89499505,3393.00,Step 1 - YMM/Manhattan


In [36]:
import pandas as pd
from datetime import datetime, timedelta
import pytz

def rename_tag_concat_and_pivot(
    cf_test_reco,
    one_to_one_test_reco,
    nonactive_test_reco,
    cf_holdout_reco,
    one_to_one_holdout_reco,
    nonactive_holdout_reco,
    cf_holdout_would_have,
    one_to_one_holdout_would_have
):
    """
    Full pipeline:
    1. Renames:
       - 'mbr_nbr' → 'input_buyer_nbr'
       - 'recommended_lot_nbr' → 'recommended_lot'
    2. Keeps ['input_buyer_nbr', 'recommended_lot']
    3. Adds 'identifier' (1=CF, 2=One-to-One, 3=Nonactive)
    4. Adds 'group' (test / holdout / would_have)
    5. Concatenates all 9 DataFrames (keeps duplicates)
    6. Reorders columns: identifier → group → input_buyer_nbr
    7. Pivots to create 6 columns for recommended lots per buyer
    8. Renames columns to lot_1 ... lot_6
    9. Converts lot columns to int
    10. Adds 'created_at' (current CST) and 'sent_at' (next day 7 AM CST)
    """

    def _rename_and_trim(df):
        rename_map = {}
        if 'mbr_nbr' in df.columns:
            rename_map['mbr_nbr'] = 'input_buyer_nbr'
        if 'recommended_lot_nbr' in df.columns:
            rename_map['recommended_lot_nbr'] = 'recommended_lot'
        df = df.rename(columns=rename_map)

        keep_cols = [col for col in ['input_buyer_nbr', 'recommended_lot'] if col in df.columns]
        return df[keep_cols]

    # --- Apply rename & trim ---
    cf_test_reco = _rename_and_trim(cf_test_reco)
    one_to_one_test_reco = _rename_and_trim(one_to_one_test_reco)
    nonactive_test_reco = _rename_and_trim(nonactive_test_reco)
    cf_holdout_reco = _rename_and_trim(cf_holdout_reco)
    one_to_one_holdout_reco = _rename_and_trim(one_to_one_holdout_reco)
    nonactive_holdout_reco = _rename_and_trim(nonactive_holdout_reco)
    cf_holdout_would_have = _rename_and_trim(cf_holdout_would_have)
    one_to_one_holdout_would_have = _rename_and_trim(one_to_one_holdout_would_have)

    # ✅ Create a separate copy for nonactive_holdout_would_have
    nonactive_holdout_would_have = nonactive_holdout_reco.copy(deep=True)
    nonactive_holdout_would_have.to_excel('../data/results/nonactive_holdout_would_have_reco.xlsx')

    # --- Add identifier and group columns ---
    mapping = [
        # TEST
        (cf_test_reco, 1, 'test'),
        (one_to_one_test_reco, 2, 'test'),
        (nonactive_test_reco, 3, 'test'),

        # HOLDOUT
        (cf_holdout_reco, 1, 'holdout'),
        (one_to_one_holdout_reco, 2, 'holdout'),
        (nonactive_holdout_reco, 3, 'holdout'),

        # WOULD_HAVE
        (cf_holdout_would_have, 1, 'would_have'),
        (one_to_one_holdout_would_have, 2, 'would_have'),
        (nonactive_holdout_would_have, 3, 'would_have')
    ]

    for df, id_val, grp in mapping:
        df['identifier'] = id_val
        df['group'] = grp

    # --- Concatenate all together ---
    combined_recos = pd.concat([
        cf_test_reco,
        one_to_one_test_reco,
        nonactive_test_reco,
        cf_holdout_reco,
        one_to_one_holdout_reco,
        nonactive_holdout_reco,
        cf_holdout_would_have,
        one_to_one_holdout_would_have,
        nonactive_holdout_would_have
    ], ignore_index=True)

    # --- Reorder columns (identifier first, group second) ---
    cols = ['identifier', 'group', 'input_buyer_nbr', 'recommended_lot']
    combined_recos = combined_recos[cols]

    # --- Rank and pivot to create 6 columns for recommended lots ---
    combined_recos['rank'] = combined_recos.groupby(['identifier', 'group', 'input_buyer_nbr']).cumcount() + 1
    pivoted = combined_recos.pivot(
        index=['identifier', 'group', 'input_buyer_nbr'],
        columns='rank',
        values='recommended_lot'
    ).reset_index()

    # ✅ Rename recommendation columns to lot_1 ... lot_6
    pivoted.columns = [
        f'lot_{int(col)}' if isinstance(col, int) else col
        for col in pivoted.columns
    ]

    # ✅ Ensure all 6 lot columns exist
    lot_cols = [f'lot_{i}' for i in range(1, 7)]
    for col in lot_cols:
        if col not in pivoted.columns:
            pivoted[col] = 0

    pivoted = pivoted[['identifier', 'group', 'input_buyer_nbr'] + lot_cols]

    # ✅ Convert lot columns to int
    pivoted[lot_cols] = pivoted[lot_cols].fillna(0).astype(int)

    # ✅ Add created_at and sent_at timestamps
    cst = pytz.timezone('US/Central')
    now_cst = datetime.now(cst)
    next_day_7am_cst = (now_cst + timedelta(days=1)).replace(hour=7, minute=0, second=0, microsecond=0)

    pivoted['created_at'] = now_cst
    pivoted['sent_at'] = next_day_7am_cst

    return pivoted


In [37]:
final_pivoted_recos = rename_tag_concat_and_pivot(
    cf_test_reco,
    one_to_one_test_reco,
    nonactive_test_reco,
    cf_holdout_reco,
    one_to_one_holdout_reco,
    nonactive_holdout_reco,
    cf_holdout_would_have,
    one_to_one_holdout_would_have
)

final_pivoted_recos.head()

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6,created_at,sent_at
0,1,holdout,408,80866645,69732105,58465645,80810675,80561425,85596785,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
1,1,holdout,738,70325995,85734915,84451805,86244335,66965175,84045605,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
2,1,holdout,866,82063205,80326835,80808925,89474305,81876055,71658605,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
3,1,holdout,1450,70469065,58894945,87367655,70803985,80320985,84825705,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
4,1,holdout,1556,80701695,59640105,82030915,66625165,80905865,69660175,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00


In [38]:
final_pivoted_recos

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6,created_at,sent_at
0,1,holdout,408,80866645,69732105,58465645,80810675,80561425,85596785,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
1,1,holdout,738,70325995,85734915,84451805,86244335,66965175,84045605,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
2,1,holdout,866,82063205,80326835,80808925,89474305,81876055,71658605,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
3,1,holdout,1450,70469065,58894945,87367655,70803985,80320985,84825705,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
4,1,holdout,1556,80701695,59640105,82030915,66625165,80905865,69660175,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...
339491,3,would_have,998374,75359674,84617715,80268005,81722415,81441775,69923105,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
339492,3,would_have,998724,65833885,75036324,70481755,67934295,67567245,86434195,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
339493,3,would_have,999222,81866185,65149575,84832845,89628655,84247235,66661425,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00
339494,3,would_have,999662,84982465,85823885,80278675,83874225,71958045,81844975,2025-10-30 11:49:45.562429-05:00,2025-10-31 07:00:00-05:00


In [39]:
final_pivoted_recos.isnull().sum()

identifier         0
group              0
input_buyer_nbr    0
lot_1              0
lot_2              0
lot_3              0
lot_4              0
lot_5              0
lot_6              0
created_at         0
sent_at            0
dtype: int64

In [40]:
final_pivoted_recos.groupby(['identifier', 'group'])['input_buyer_nbr'].nunique()

identifier  group     
1           holdout        3694
            test           3618
            would_have     3694
2           holdout       11258
            test          11250
            would_have    11258
3           holdout       98312
            test          98100
            would_have    98312
Name: input_buyer_nbr, dtype: int64

In [42]:
from datetime import datetime, timedelta
import pytz

# Get current CST date
cst = pytz.timezone('US/Central')
now_cst = datetime.now(cst)

# Get tomorrow's date in CST (YYYY-MM-DD format)
tomorrow_date = (now_cst + timedelta(days=1)).strftime("%Y-%m-%d")

# Build the file path
file_path = f"../data/final/recommendations_{tomorrow_date}.xlsx"

# 🔧 Remove timezone from any datetime columns
for col in final_pivoted_recos.select_dtypes(include=['datetimetz']).columns:
    final_pivoted_recos[col] = final_pivoted_recos[col].dt.tz_localize(None)

# Save DataFrame
final_pivoted_recos.to_excel(file_path, index=False)

print(f"✅ File saved successfully as: {file_path}")


✅ File saved successfully as: ../data/final/recommendations_2025-10-31.xlsx


In [43]:
from google.cloud import bigquery

def upload_to_bigquery(dataframe, table_id, project_id, credentials_path):
    # Initialize BigQuery client
    client = bigquery.Client.from_service_account_json(credentials_path)

    # Define job configuration
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # Appends data to the table if it exists
        autodetect=True,  # Automatically detects schema
    )

    # Upload the DataFrame to BigQuery
    job = client.load_table_from_dataframe(dataframe, table_id, job_config=job_config)

    # Wait for the job to complete
    job.result()

    print(f"Data appended to {table_id} in project {project_id}.")

# Example usage
upload_to_bigquery(
    dataframe=final_pivoted_recos,  # Replace with your DataFrame
    table_id="member_reco.member_future_reco",  # Replace with your dataset and table name
    project_id="cprtqa_strategicanalytics-sp1",  # Replace with your GCP project ID
    credentials_path='/Users/srdeo/OneDrive - Copart, Inc/cprtqa-strategicanalytics-sp1-8b7a00c4fbae.json'  # Replace with your credentials file path
)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Data appended to member_reco.test_future_reco in project cprtqa_strategicanalytics-sp1.
